In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score
import folium

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['SEC_ORDER_VISUAL']['LON_MIN'])
lonMax = (float)(config['SEC_ORDER_VISUAL']['LON_MAX'])

latMin = (float)(config['SEC_ORDER_VISUAL']['LAT_MIN'])
latMax = (float)(config['SEC_ORDER_VISUAL']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['SEC_ORDER_VISUAL']['INCR_STEP'])
incrRes = (int)(config['SEC_ORDER_VISUAL']['INCR_RES'])

boundary = (float)(config['SEC_ORDER_VISUAL']['BOUNDARY'])

fileDir = config['TP_SEC_ORDER']['SOURCE_DIR']
dirToStore = config['TP_SEC_ORDER']['DEST_DIR']
# destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj1/"
# destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj2/"
# destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj3/"
# destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj4/"
# destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj5/"
# destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj6/"
destDirTraj = "../Data/M120_50_M119_00_33_90_34_44/Display/Traj7/"

In [ ]:
#this is useful for mapping
#probabilistic location
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
lonCen = (lonMin + lonMax)/2
latCen = (latMin + latMax)/2
print(lonCen)
print(latCen)


In [ ]:
#Generate trajectory list
#for one vessel
MMSI_DIR = "../Data/M120_50_M119_00_33_90_34_44/MMSI/"
def get_vessel_data(vesselName):
    oneVessel = MMSI_DIR + vesselName +"_Sorted.csv"
    oneVesselData,_ = aISDM.load_data_from_csv(oneVessel)
    return oneVesselData

def get_vessel_lon_lat_from_traj(vesselName):
    oneVesselData = get_vessel_data(vesselName)
    print(oneVesselData.shape)
    oneVesselLon = oneVesselData['LON']
    oneVesselLat = oneVesselData['LAT']
    return oneVesselLon, oneVesselLat

    
oneVesselData = get_vessel_data("565819000")
oneVesselLon,oneVesselLat = get_vessel_lon_lat_from_traj("565819000")

In [ ]:
oneVesselTime = oneVesselData['DateTime']
for i in range(oneVesselLon.shape[0]):
    print(i,oneVesselTime[i])

In [ ]:
#for trajectory 1 and 2
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1704To1705_00.txt"
# timeIntervalList1 = "../Data/TimeInterval/HalfHourIntvl1704To1705_05.txt"
# timeIntervalList2 = "../Data/TimeInterval/HalfHourIntvl1704To1705_10.txt"
# timeIntervalList3 = "../Data/TimeInterval/HalfHourIntvl1704To1705_15.txt"
# timeIntervalList4 = "../Data/TimeInterval/HalfHourIntvl1704To1705_20.txt"
# timeIntervalList5 = "../Data/TimeInterval/HalfHourIntvl1704To1705_25.txt"

#for trajectory 3

# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1706To1707_00.txt"
# timeIntervalList1 = "../Data/TimeInterval/HalfHourIntvl1706To1707_05.txt"
# timeIntervalList2 = "../Data/TimeInterval/HalfHourIntvl1706To1707_10.txt"
# timeIntervalList3 = "../Data/TimeInterval/HalfHourIntvl1706To1707_15.txt"
# timeIntervalList4 = "../Data/TimeInterval/HalfHourIntvl1706To1707_20.txt"
# timeIntervalList5 = "../Data/TimeInterval/HalfHourIntvl1706To1707_25.txt"

#for trajectory 4 and 5
# timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1707To1708_00.txt"
# timeIntervalList1 = "../Data/TimeInterval/HalfHourIntvl1707To1708_05.txt"
# timeIntervalList2 = "../Data/TimeInterval/HalfHourIntvl1707To1708_10.txt"
# timeIntervalList3 = "../Data/TimeInterval/HalfHourIntvl1707To1708_15.txt"
# timeIntervalList4 = "../Data/TimeInterval/HalfHourIntvl1707To1708_20.txt"
# timeIntervalList5 = "../Data/TimeInterval/HalfHourIntvl1707To1708_25.txt"

#for trajectory 6
timeIntervalList = "../Data/TimeInterval/HalfHourIntvl1708To1709_00.txt"
timeIntervalList1 = "../Data/TimeInterval/HalfHourIntvl1708To1709_05.txt"
timeIntervalList2 = "../Data/TimeInterval/HalfHourIntvl1708To1709_10.txt"
timeIntervalList3 = "../Data/TimeInterval/HalfHourIntvl1708To1709_15.txt"
timeIntervalList4 = "../Data/TimeInterval/HalfHourIntvl1708To1709_20.txt"
timeIntervalList5 = "../Data/TimeInterval/HalfHourIntvl1708To1709_25.txt"

timeWindows = [line.rstrip('\n') for line in open(timeIntervalList)]
timeWindows1 = [line.rstrip('\n') for line in open(timeIntervalList1)]
timeWindows2 = [line.rstrip('\n') for line in open(timeIntervalList2)]
timeWindows3 = [line.rstrip('\n') for line in open(timeIntervalList3)]
timeWindows4 = [line.rstrip('\n') for line in open(timeIntervalList4)]
timeWindows5 = [line.rstrip('\n') for line in open(timeIntervalList5)]

In [ ]:
#get the last instance of the position data 
#for one of the vessel
#and that can be used for the prediction
oneVesselLastData = pd.DataFrame()
oneVesselLastData1 = pd.DataFrame()
oneVesselLastData2 = pd.DataFrame()
oneVesselLastData3 = pd.DataFrame()
oneVesselLastData4 = pd.DataFrame()
oneVesselLastData5 = pd.DataFrame()
#based on time stamps 
for timeSlot in timeWindows:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData = oneVesselLastData.append(invertedTimelyDF, ignore_index = True)
    
for timeSlot in timeWindows1:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData1 = oneVesselLastData1.append(invertedTimelyDF, ignore_index = True)
    
for timeSlot in timeWindows2:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData2 = oneVesselLastData2.append(invertedTimelyDF, ignore_index = True)
    
for timeSlot in timeWindows3:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData3 = oneVesselLastData3.append(invertedTimelyDF, ignore_index = True)
    
for timeSlot in timeWindows4:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData4 = oneVesselLastData4.append(invertedTimelyDF, ignore_index = True)
    
for timeSlot in timeWindows5:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData5 = oneVesselLastData5.append(invertedTimelyDF, ignore_index = True)

In [ ]:
print("**************0")
print(oneVesselLastData)
print("**************1")
print(oneVesselLastData1)
print("**************2")
print(oneVesselLastData2)
print("**************3")
print(oneVesselLastData3)
print("**************4")
print(oneVesselLastData4)
print("**************5")
print(oneVesselLastData5)

In [ ]:
#for traj1
# oneVesselLastDataOne = oneVesselLastData.iloc[0:15,:]
# oneVesselLastDataOne1 = oneVesselLastData1.iloc[0:15,:]
# oneVesselLastDataOne2 = oneVesselLastData2.iloc[0:16,:]
# oneVesselLastDataOne3 = oneVesselLastData3.iloc[0:16,:]
# oneVesselLastDataOne4 = oneVesselLastData4.iloc[0:16,:]
# oneVesselLastDataOne5 = oneVesselLastData5.iloc[0:16,:]

#for traj2
# oneVesselLastDataOne = oneVesselLastData.iloc[15:,:]
# oneVesselLastDataOne1 = oneVesselLastData1.iloc[15:,:]
# oneVesselLastDataOne2 = oneVesselLastData2.iloc[16:,:]
# oneVesselLastDataOne3 = oneVesselLastData3.iloc[16:,:]
# oneVesselLastDataOne4 = oneVesselLastData4.iloc[16:,:]
# oneVesselLastDataOne5 = oneVesselLastData5.iloc[16:,:]

#for traj3
# oneVesselLastDataOne = oneVesselLastData.iloc[1:,:]
# oneVesselLastDataOne1 = oneVesselLastData1.iloc[1:,:]
# oneVesselLastDataOne2 = oneVesselLastData2.iloc[1:,:]
# oneVesselLastDataOne3 = oneVesselLastData3.iloc[1:,:]
# oneVesselLastDataOne4 = oneVesselLastData4.iloc[1:,:]
# oneVesselLastDataOne5 = oneVesselLastData5.iloc[1:,:]

#for traj4
# oneVesselLastDataOne = oneVesselLastData.iloc[1:11,:]
# oneVesselLastDataOne1 = oneVesselLastData1.iloc[1:10,:]
# oneVesselLastDataOne2 = oneVesselLastData2.iloc[1:10,:]
# oneVesselLastDataOne3 = oneVesselLastData3.iloc[1:11,:]
# oneVesselLastDataOne4 = oneVesselLastData4.iloc[1:11,:]
# oneVesselLastDataOne5 = oneVesselLastData5.iloc[1:11,:]

#for traj5
# oneVesselLastDataOne = oneVesselLastData.iloc[13:,:]
# oneVesselLastDataOne1 = oneVesselLastData1.iloc[12:,:]
# oneVesselLastDataOne2 = oneVesselLastData2.iloc[12:,:]
# oneVesselLastDataOne3 = oneVesselLastData3.iloc[13:,:]
# oneVesselLastDataOne4 = oneVesselLastData4.iloc[13:,:]
# oneVesselLastDataOne5 = oneVesselLastData5.iloc[13:,:]

#for traj6
oneVesselLastDataOne = oneVesselLastData.iloc[5:,:]
oneVesselLastDataOne1 = oneVesselLastData1.iloc[5:,:]
oneVesselLastDataOne2 = oneVesselLastData2.iloc[5:,:]
oneVesselLastDataOne3 = oneVesselLastData3.iloc[5:,:]
oneVesselLastDataOne4 = oneVesselLastData4.iloc[5:,:]
oneVesselLastDataOne5 = oneVesselLastData5.iloc[5:,:]

In [ ]:
print("*************0")
print(oneVesselLastDataOne)
print("*************1")
print(oneVesselLastDataOne1)
print("*************2")
print(oneVesselLastDataOne2)
print("*************3")
print(oneVesselLastDataOne3)
print("*************4")
print(oneVesselLastDataOne4)
print("*************5")
print(oneVesselLastDataOne5)

In [ ]:
traj = []
#for traj 1
# traj.append(oneVesselLastDataOne2)
# traj.append(oneVesselLastDataOne3)
# traj.append(oneVesselLastDataOne4)
# traj.append(oneVesselLastDataOne5)
# traj.append(oneVesselLastDataOne)
# traj.append(oneVesselLastDataOne1)

#for traj 2
# traj.append(oneVesselLastDataOne3)
# traj.append(oneVesselLastDataOne4)
# traj.append(oneVesselLastDataOne5)
# traj.append(oneVesselLastDataOne)
# traj.append(oneVesselLastDataOne1)
# traj.append(oneVesselLastDataOne2)

#for traj 3
# traj.append(oneVesselLastDataOne3)
# traj.append(oneVesselLastDataOne4)
# traj.append(oneVesselLastDataOne5)
# traj.append(oneVesselLastDataOne)
# traj.append(oneVesselLastDataOne1)
# traj.append(oneVesselLastDataOne2)

#for traj 4
# traj.append(oneVesselLastDataOne3)
# traj.append(oneVesselLastDataOne4)
# traj.append(oneVesselLastDataOne5)
# traj.append(oneVesselLastDataOne)
# traj.append(oneVesselLastDataOne1)
# traj.append(oneVesselLastDataOne2)

#for traj 5
# traj.append(oneVesselLastDataOne)
# traj.append(oneVesselLastDataOne1)
# traj.append(oneVesselLastDataOne2)
# traj.append(oneVesselLastDataOne3)
# traj.append(oneVesselLastDataOne4)
# traj.append(oneVesselLastDataOne5)

#for traj 6
traj.append(oneVesselLastDataOne2)
traj.append(oneVesselLastDataOne3)
traj.append(oneVesselLastDataOne4)
traj.append(oneVesselLastDataOne5)
traj.append(oneVesselLastDataOne)
traj.append(oneVesselLastDataOne1)

In [ ]:
#get the maximum number of rows
maxRows = max(oneVesselLastDataOne.shape[0] \
              , oneVesselLastDataOne1.shape[0] \
              , oneVesselLastDataOne2.shape[0] \
              , oneVesselLastDataOne3.shape[0] \
              , oneVesselLastDataOne4.shape[0] \
              , oneVesselLastDataOne5.shape[0] \
             )

print(maxRows)

In [ ]:
interpolatedTraj = pd.DataFrame()
for i in range(maxRows):
    if(i < traj[0].shape[0]):
        interpolatedTraj = interpolatedTraj.append(traj[0].iloc[i,:], ignore_index = True)
    if(i < traj[1].shape[0]):
        interpolatedTraj = interpolatedTraj.append(traj[1].iloc[i,:], ignore_index = True)
    if(i < traj[2].shape[0]):
        interpolatedTraj = interpolatedTraj.append(traj[2].iloc[i,:], ignore_index = True)
    if(i < traj[3].shape[0]):
        interpolatedTraj = interpolatedTraj.append(traj[3].iloc[i,:], ignore_index = True)
    if(i < traj[4].shape[0]):
        interpolatedTraj = interpolatedTraj.append(traj[4].iloc[i,:], ignore_index = True)
    if(i < traj[5].shape[0]):
        interpolatedTraj = interpolatedTraj.append(traj[5].iloc[i,:], ignore_index = True)

In [ ]:
for i in range(interpolatedTraj.shape[0]):
    print(interpolatedTraj.iloc[i,0])

In [ ]:
opFile = dirToStore + 'TPM.npz'
# opFile = dirToStore + 'TPM_.npz'
tpMatrix = scipy.sparse.load_npz(opFile)

In [ ]:
print(tpMatrix.sum())

In [ ]:
def get_vessel_init_status(lonStart,latStart):
    vesselState = np.zeros((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    for boundary in boundaryArray :
        if((lonStart >= boundary[0]) \
            and (lonStart < boundary[1]) \
           
            and (latStart >= boundary[2]) \
            and (latStart < boundary[3]) \
          ):
            vesselState[boundary[4]] = 1.0
            break
    return vesselState

#takes inital state vector
#and number of steps 
#uses transition probabilities to generate the path
def generate_path(vesselStatus):
    return (vesselStatus.T @ tpMatrix).flatten()

def generate_vessel_prev_cur_status(prevState,currState):
    vesselState = np.zeros((totalStates*totalStates))
    stateCounter = 0
    for i in range(prevState.shape[0]):
        for j in range(currState.shape[0]):
            vesselState[stateCounter] = prevState[i]*currState[j]
            stateCounter = stateCounter + 1
    return vesselState

def generate_vessel_prev_cur_status_(prevState,currState):
    vesselState = np.zeros((prevState.shape[0]*currState.shape[0]))
    stateCounter = 0
    for i in range(prevState.shape[0]):
        for j in range(currState.shape[0]):
            vesselState[stateCounter] = prevState[i]*currState[j]
            stateCounter = stateCounter + 1
    return vesselState

In [ ]:
rowSum = np.sum(tpMatrix, axis=1)
rowSumZeroIdx = np.argwhere(rowSum == 0)
rowSumZeroIdx = rowSumZeroIdx.flatten()

In [ ]:
#takesdata frame
#extracts lon and lat
#returns prediction list
def generate_prediction_list(dFObj):
    oneVesselLastDataOneLon = dFObj["LON"]
    oneVesselLastDataOneLat = dFObj["LAT"]
    
    onePredict = []
    vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
    onePredict.append(vesselPrevState.copy())
    vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
    onePredict.append(vesselCurrState.copy())
    vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
    for i in range(dFObj.shape[0]-2):
        vesselProbState = generate_path(vesselState)
        print(i)
        print(np.sum(vesselProbState))
        onePredict.append(vesselProbState.copy())
        vesselPrevState = vesselCurrState.copy()
        vesselCurrState = vesselProbState.copy()
        vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
        #make probabilities zero
        vesselState[rowSumZeroIdx] = 0
        vesselState = vesselState/np.sum(vesselState)
        print(np.sum(vesselState))
    return onePredict

In [ ]:
onePredict0 = generate_prediction_list(traj[0])
onePredict1 = generate_prediction_list(traj[1])
onePredict2 = generate_prediction_list(traj[2])
onePredict3 = generate_prediction_list(traj[3])
onePredict4 = generate_prediction_list(traj[4])
onePredict5 = generate_prediction_list(traj[5])

In [ ]:
interpolatedProbTraj = []
for i in range(maxRows):
    if(i < len(onePredict0)):
        interpolatedProbTraj.append(onePredict0[i])
    if(i < len(onePredict1)):
        interpolatedProbTraj.append(onePredict1[i])
    if(i < len(onePredict2)):
        interpolatedProbTraj.append(onePredict2[i])
    if(i < len(onePredict3)):
        interpolatedProbTraj.append(onePredict3[i])
    if(i < len(onePredict4)):
        interpolatedProbTraj.append(onePredict4[i])
    if(i < len(onePredict5)):
        interpolatedProbTraj.append(onePredict5[i])

In [ ]:
print(len(onePredict0))
print(len(onePredict1))
print(len(onePredict2))
print(len(onePredict3))
print(len(onePredict4))
print(len(onePredict5))

In [ ]:
print(interpolatedProbTraj)

In [ ]:
for i in range(len(interpolatedProbTraj)):
    opFile = destDirTraj + str(i) +'.npy'
    np.save(opFile, interpolatedProbTraj[i])

In [ ]:
print(interpolatedTraj.shape)

In [ ]:
aISDM.save_data_to_csv(interpolatedTraj,(destDirTraj+"True.csv"))